# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import nltk
from sqlalchemy import *
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('Disaster',engine)
X = df['message']
Y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex,text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,"urlplaceholder")
        
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip().lower()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf',MultiOutputClassifier(KNeighborsClassifier(n_neighbors=5)))
            ])


        



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x00000282AA2058C8>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='au

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
target_names = df.iloc[:,4:].columns

In [9]:
for i in range(y_pred.shape[1]):
    print("Label- {} ".format(target_names[i]))
    print(classification_report(y_test[:,i], y_pred[:,i]))
    print("")

Label- related 
              precision    recall  f1-score   support

           0       0.54      0.39      0.46      1502
           1       0.83      0.90      0.86      5005
           2       0.70      0.34      0.46        47

    accuracy                           0.78      6554
   macro avg       0.69      0.54      0.59      6554
weighted avg       0.76      0.78      0.77      6554


Label- request 
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      5403
           1       0.77      0.33      0.46      1151

    accuracy                           0.87      6554
   macro avg       0.82      0.65      0.69      6554
weighted avg       0.86      0.87      0.84      6554


Label- offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50   

c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.96      6143
           1       0.22      0.03      0.06       411

    accuracy                           0.93      6554
   macro avg       0.58      0.51      0.51      6554
weighted avg       0.89      0.93      0.91      6554


Label- transport 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6286
           1       0.47      0.07      0.12       268

    accuracy                           0.96      6554
   macro avg       0.72      0.53      0.55      6554
weighted avg       0.94      0.96      0.94      6554


Label- buildings 
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6256
           1       0.35      0.07      0.11       298

    accuracy                           0.95      6554
   macro avg       0.65      0.53      0.54      6554
weighted avg       0.93      0.95   

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__n_neighbors':[3,5,11,19],
    'clf__estimator__weights':['uniform','distance'],
    'clf__estimator__metric':['euclidean','manhattan']
    
}
clf = GridSearchCV(pipeline, parameters, n_jobs=-1)

In [12]:
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [15]:
!pip install joblib

In [19]:
import joblib
joblib.dump(clf, 'model.pkl')

['model.pkl']

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred = clf.predict(X_test)
for i in range(y_pred.shape[1]):
    print("Label- {} ".format(target_names[i]))
    print(classification_report(y_test[:,i], y_pred[:,i]))
    print("")

Label- related 
              precision    recall  f1-score   support

           0       0.48      0.46      0.47      1502
           1       0.84      0.85      0.84      5005
           2       0.36      0.34      0.35        47

    accuracy                           0.76      6554
   macro avg       0.56      0.55      0.55      6554
weighted avg       0.75      0.76      0.75      6554


Label- request 
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5403
           1       0.76      0.10      0.18      1151

    accuracy                           0.84      6554
   macro avg       0.80      0.55      0.54      6554
weighted avg       0.82      0.84      0.78      6554


Label- offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50   

c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6489
           1       0.00      0.00      0.00        65

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554


Label- shops 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554


Label- aid_centers 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.00      0.00      0.00        76

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [6]:
from sklearn.linear_model import Perceptron
from sklearn.multiclass import OneVsRestClassifier
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf',MultiOutputClassifier(OneVsRestClassifier(Perceptron(tol=1e-3, random_state=0))))
            ])


In [7]:
pipeline.fit(X_train, y_train)

C:\Users\PANKHIL\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=Perceptron(alpha=0.0001,
                                                                                          class_weight=None,
                                                                                          early_st

In [35]:
import joblib
filename = 'test.sav'
joblib.dump(pipeline, filename)


['test.sav']

In [36]:
loaded_model = joblib.load('test.sav')

In [37]:
s = loaded_model.predict([''])

In [38]:
len(s[0])

36

In [39]:
ind = np.where(s[0]==1)

In [40]:
ind

(array([ 0,  1,  3, 10, 11, 12, 16, 26, 35], dtype=int64),)

In [42]:
for k in ind:
    print(target_names[k])

Index(['related', 'request', 'aid_related', 'water', 'food', 'shelter',
       'refugees', 'aid_centers', 'direct_report'],
      dtype='object')


In [18]:
Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)
category_names = Y.columns

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [81]:
y_pred = pipeline.predict(X_test)

In [82]:
for i in range(y_pred.shape[1]):
    print("Label- {} ".format(category_names[i]))
    print(classification_report(y_test[:,i], y_pred[:,i],))
    print("")

Label- related 
              precision    recall  f1-score   support

           0       0.57      0.48      0.52      1589
           1       0.84      0.88      0.86      4930
           2       0.44      0.43      0.43        35

    accuracy                           0.78      6554
   macro avg       0.61      0.60      0.60      6554
weighted avg       0.77      0.78      0.77      6554


Label- request 
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      5499
           1       0.58      0.49      0.53      1055

    accuracy                           0.86      6554
   macro avg       0.74      0.71      0.73      6554
weighted avg       0.85      0.86      0.86      6554


Label- offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.06      0.03      0.04        40

    accuracy                           0.99      6554
   macro avg       0.53   

In [33]:
import joblib
joblib.dump(clf, 'Percentron.pkl')

['Percentron.pkl']

In [26]:

parameters = {
    'clf__estimator__estimator__alpha':[0.0003, 0.001, 0.003, 0.01, 0.1],
    'clf__estimator__estimator__n_iter_no_change':[5, 10, 15, 20, 50]

}
clf = GridSearchCV(pipeline, parameters,verbose=3,cv=3)

In [27]:
clf.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] clf__estimator__estimator__alpha=0.0003, clf__estimator__estimator__n_iter_no_change=5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  clf__estimator__estimator__alpha=0.0003, clf__estimator__estimator__n_iter_no_change=5, score=0.171, total=   8.2s
[CV] clf__estimator__estimator__alpha=0.0003, clf__estimator__estimator__n_iter_no_change=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s remaining:    0.0s
c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  clf__estimator__estimator__alpha=0.0003, clf__estimator__estimator__n_iter_no_change=5, score=0.174, total=   7.8s
[CV] clf__estimator__estimator__alpha=0.0003, clf__estimator__estimator__n_iter_no_change=5 


...

[CV]  clf__estimator__estimator__alpha=0.1, clf__estimator__estimator__n_iter_no_change=50, score=0.168, total=  11.2s
[CV] clf__estimator__estimator__alpha=0.1, clf__estimator__estimator__n_iter_no_change=50 


c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


[CV]  clf__estimator__estimator__alpha=0.1, clf__estimator__estimator__n_iter_no_change=50, score=0.163, total=  11.7s


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 11.1min finished
c:\users\pankhil\appdata\local\programs\python\python36\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 9. Export your model as a pickle file

In [1]:
import joblib


In [38]:
per_cv = joblib.load('Percentron.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.